# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import Workspace, Experiment, Dataset, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.run import Run

import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
#Retrieve current workspace
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Tuscan_Wine_Forecast'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')



## Setting up Environment

In [ ]:
env = Environment(name="AzureML-Tutorial")

## Compute Cluster
If compute cluster already exists, it will just identify it and end.


In [ ]:
#Create Cluster if does not already exist

cluster_name = "myCluster"
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Cluster already created")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",min_nodes=1, max_nodes=6)
    cluster = ComputeTarget.create(ws,cluster_name, compute_config) #creates the actual cluster

cluster.wait_for_completion(show_output=True)  #Allows to continue on other threads while cluster is being created


#SOURCE / HELP: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute.amlcompute?view=azure-ml-py

## Dataset
A Dataset is created from the csv file in the folder "data".

In [ ]:
# Create a dataset from delimited files with header option as ONLY_FIRST_FILE_HAS_HEADERS

   datastore = ws.get_default_datastore() 
   dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'data/dataset_Tuscany_Wine_prepared.csv'),
       header='ONLY_FIRST_FILE_HAS_HEADERS')

   df = dataset.to_pandas_dataframe()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
 
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=5)
#Ends runs when the primary metric isn't above (1/(1+0,1) ~90%. Start evaluating after 5 intervals.

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
        "--C": uniform(0.1, 1),
        "--max_iter": choice(5,10,25,100,1000)
})
#Random / Grid  https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters?view=azure-ml-py
#Bayesian sampling does not support early termination



#Logging metrics


run_logger = Run.get_context()
run_logger.log("accuracy", float(val_accuracy))


#TODO: Create your estimator and hyperdrive config
#estimator = SKLearn(
    #source_directory=os.path.join('./'),
    #compute_target=cluster,
    #entry_script="train.py"
#)

script_run_config = ScriptRunConfig(
    source_directory=os.path.join('./'),
    compute_target=cluster,
    entry_script="train.py",
    environment = env
)


hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                        policy = early_termination_policy,
                                        primary_metric_name="accuracy",
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                        ax_duration_minutes = 30,
                                        max_concurrent_runs= 5,
                                        #estimator= estimator
                                        run_config=script_run_config)

In [ ]:
#TODO: Submit your experiment

experiment = Experiment(ws, experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

